In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('../../results/result.csv')

repo_num = 3042
# merge_tool = "gitmerge-recursive-patience"
# merge_tool = "gitmerge-recursive-minimal"
merge_tool = "spork"
# merge_tool = "gitmerge-resolve"
# merge_tool = "intellimerge"
# merge_tool = "gitmerge-ort-ignorespace"
# merge_tool = "gitmerge-ort"

script = "../scripts/merge_tools/" + merge_tool + ".sh"

In [ ]:
from merge_tester import MERGE_STATE
df[merge_tool].apply(lambda x: MERGE_STATE[x])[0]

In [ ]:
from validate_repos import clone_repo
repo = clone_repo(df.iloc[repo_num]["repo_name"]) # Return a Git-Python repo object

In [ ]:
# For a git to work you need to pass a name for each branch
repo.remote().fetch()
repo.git.checkout(df.iloc[repo_num]["left"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_LEFT_BRANCH", force=True)
repo.git.checkout(df.iloc[repo_num]["right"], force=True)
repo.submodule_update()
repo.git.checkout("-b", "TEMP_RIGHT_BRANCH", force=True)

In [ ]:
repo_path = repo.git.rev_parse("--show-toplevel")
repo_path

In [ ]:
import subprocess
# Returns = 1 process fails
subprocess.call([script,repo_path,"TEMP_LEFT_BRANCH","TEMP_RIGHT_BRANCH"])

In [ ]:
subprocess.call(["cd",repo_path,"&&","git","merge","--abort"])

In [ ]:
# Merge Sucess (Return 0)
subprocess.call(["../scripts/merge_tools/gitmerge-ort-ignorespace.sh",repo_path,"NAME_OF_LEFT_BRANCH","NAME_OF_RIGHT_BRANCH"])
# Be careful the merge has been completed the history of the left branch has now changed and you can't 
# merge again unless you delete the repo and clone it again